# Анализ данных и визуализация информации

## Подготовка данных

Ниже приведен код, который упрощает повторную обработку датасета. Данный набор данных мы будем использовать дальше в учебных целях.

```python
import pandas as pd
import numpy as np
import sqlalchemy

engine = sqlalchemy.create_engine(
                "mysql+pymysql://root:__PASS__@__IP__6:3307/rzd", encoding='utf8', convert_unicode=True
            )

with engine.connect() as session:
    df=pd.read_sql('SELECT * FROM ku_asrb LIMIT 1000', con=session)
    
columns_date=['Дата события', 'Дата создания события в системе AC PБ', ]
columns_float=[ 'Размер возмещенного ущерба (тыс.руб.)',
               'Итоговый суммарный ущерб (тыс.руб.)']
columns_int=['Погибло всего', 'Погибло пассажиров',
       'Погибло сторонних', 'Погибло прочих',
       'Получили тяжкие телесные повреждения: сотрудников ОАО "РЖД"',
       'Получили тяжкие телесные повреждения: пассажиров',
       'Получили тяжкие телесные повреждения: сторонние',
       'Получили тяжкие телесные повреждения: прочие', 'Ранено легко', 
       'Станция/перегон id', 'Переезд', 'Путь общего/необщего пользования',
       'Номер пути', 'Километр', 'Пикет', 'Общее время задержки']
columns_time=['Время полного перерыва движения', 'Время расстройства маневровой работы',
              'Количество задержанных поездов']

def ddate(s):
    try:
        res=pd.to_datetime(s, format='%d%b%Y:%H:%M:%S')
    except:
        res=np.NaN
    return res

for i in columns_date:
    df[i]=df[i].apply(lambda x: ddate(x))
    
for i in columns_float:
    df.loc[df[i]=='.', i]='0'
    df[i]=df[i].astype(float)
    
for i in columns_int:
    df[i].fillna('.', inplace=True)
    df.loc[df[i]=='.', i]='0'
    df[i]=df[i].astype(int)
    
def time_to_sec(s):
    s3=s.split(':')
    if len(s3)<3:
        return 0
    else:
        return int(s3[0])*60*60+int(s3[1])*60+int(s3[2])
    
for i in columns_time:
    df[i]=df[i].apply(lambda x: time_to_sec(x))
    
df['Время до регистрации']=df.apply(lambda x:(x['Дата создания события в системе AC PБ']-x['Дата события']).total_seconds(), axis=1)

df.info()
```

# Разведывательный анализ данных EDA

Выберем несколько колонок для примера.

```python
col=['Итоговый суммарный ущерб (тыс.руб.)', 'Общее время задержки',
       'Количество задержанных поездов', 'Время до регистрации']
df[col].describe()
```

Построим гистограммы.

```python
df[col].hist(figsize=(15,5), bins=40);
```

Выведем события выше некоторого уровня, чтобы более внимательно их изучить.

```python
df[df['Общее время задержки']>30]
```

Строки с большими значениями (выбросы), могут мешать рассмотреть тенденции в основном массиве данных. Их стоит часто удалять или заменить на более приемлемые значения.

Несколько графиков для визуализации данных.

```python
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

plt.figure(figsize=(10,10))
sns.boxplot(data=df[col], palette='rainbow', orient='h');
```

Парные материцы рассеяния.

```python
sns.pairplot(df[col],height=3);
```

Укрупненный срез.

```python
sns.jointplot(x='Общее время задержки', y='Итоговый суммарный ущерб (тыс.руб.)', data=df, kind='scatter');
```

При разведочном анализи стоит посмотреть на взаимосвязь между переменными. А именно, на корреляцию.

```python
df[col].corr()
```

И построить тепловую карту для лучшего восприятия.

```python
corr = df[col].corr()
plt.figure(figsize=(14, 14))
sns.heatmap(corr[(corr >= 0.3) | (corr <= -0.3)],
            cmap="RdBu_r", vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);
```

Вариант тепловой карты с группировкой по наиболее тесной взаимосвязи.

```python
plt.figure(figsize=(14,14))
sns.clustermap(df[col].corr())
```

Помимо корреляции надо обращать внимание и на размах величины, а именно на ее коэффициент вариации.

```python
from scipy.stats import variation
pd.DataFrame(variation(df[col]), index=col)
````

Более нагляно об идее.

```python
df2=df[col]/df[col].max()
df2.plot(figsize=(17,5))
```

При формировании гипотез полезно посмотреть и на группировки по различным качественным признакам, понять как они влияют на показатели.

```python
pd.options.display.float_format = '{:,.2f}'.format
df.groupby('Код дороги')[col].mean()
```

### Обработка выбросов

Давайте вернемся к выбросам. Мы увидели, что они очень сильно портят восприятие данных аналитиком. 

Выбросы - это значения, которые намного превышают следующие ближайшие точки данных.

Есть два типа выбросов:
- Одномерные выбросы: Одномерные выбросы - это точки данных, значения которых выходят за пределы диапазона ожидаемых значений, основанных на одной переменной.
- Многовариантные выбросы: при построении данных некоторые значения одной переменной могут не выходить за пределы ожидаемого диапазона, но при отображении данных с какой-либо другой переменной эти значения могут находиться далеко от ожидаемого значения.

Аналитик основываясь на своем опыте должен принять решение, как поступить с выбросами. Заменить, удалить или оставить. В нашем случае, мы удалим выбросы.

Следующие операции выполним последовательно.

```python
df[col].describe()

df['Итоговый суммарный ущерб (тыс.руб.)'].hist(figsize=(14,4), bins=50)

sns.boxplot(df[df['Итоговый суммарный ущерб (тыс.руб.)']<50]['Итоговый суммарный ущерб (тыс.руб.)']);

df[df['Итоговый суммарный ущерб (тыс.руб.)']<50]['Итоговый суммарный ущерб (тыс.руб.)'].hist(figsize=(14,4), bins=50);

df.drop(df[df['Итоговый суммарный ущерб (тыс.руб.)']>50].index, inplace=True, axis='index')
```
Проделаем такие же операции и с другими параметрами. Но, например, в случае общего времени задержки, всем выбросам присвоим некоторое максимальное значение.

# Нормирование данные

Для большинства алгоритмов машинного обучения надо выполнить нормирование данные. Нормирование данных заключается в приведении диапазона изменения значений признаков к некоторым требуемым границам (например, от 0 до 1). Нормирование является необходимым начальным этапом обработки данных при использовании многих многомерных статистических методов — снижения размерности признакового пространства, классификации и т.д., особенно если переменные измерены в шкалах, существенно различающихся в величинах (например, от миллиметров до километров).

Воспользуемся препроцессиногом библиотеки sklearn.

```python
from sklearn import preprocessing

import random
import numpy as np
import matplotlib.pyplot as plt

a=np.array([[random.randint(0,10) for i in range(2)] for z in range(20)])
a
```

МинМакс скаллерл лучше всего подходит для отображения данных и их использования при той же кластеризации.

```python
min_max_scaler = preprocessing.MinMaxScaler()
minmax = min_max_scaler.fit_transform(a)
minmax
```


StandardScaler лучше подходжит для номирования в рамках алгоритмов машинного обучения. 

```python
scaler = preprocessing.StandardScaler().fit(a)
standart=scaler.fit_transform(a)
standart
```

RobustScaler лучше работает с зашумленными данными и выбросами.

```python
from sklearn.preprocessing import RobustScaler
robust = RobustScaler().fit_transform(a)
robust
```

Для того, чтобы лучше понять, как работают различные способы нормирования, отобразим их на графике.

```python
import seaborn as sns
import matplotlib.pyplot as plt

f=np.array([a, standart, minmax, robust])
label=['Оригинальные', 'Standart', 'MinMax', 'Robust']

fig, axs = plt.subplots(1, 4, figsize=(15, 4))
plt.title("title")
for i, ax in enumerate(axs):
    x=f[i,:,0]
    y=f[i,:,1]
    ax.title.set_text(label[i])
    ax.scatter(x, y)
    ax.grid()
```

Или более наглядно.

```python
import seaborn as sns

m=[]

for i in range(len(f)):
    for j in f[i]:
        m.append([j[0], j[1], label[i]])
dfG=pd.DataFrame(m, columns=['x','y','standart'])

fig, ax = plt.subplots(figsize=(8,6))
sns.scatterplot(x='x', y='y', hue='standart', data=dfG) 
plt.show()
```

# Кластеризация

Обучение без учителя (unsupervised learning, неконтролируемое обучение) – класс методов машинного обучения для поиска шаблонов в наборе данных. Данные, получаемые на вход таких алгоритмов, обычно не размечены, то есть передаются только входные переменные X без соответствующих меток y. Если в контролируемом обучении (обучении с учителем, supervised learning) система пытается извлечь уроки из предыдущих примеров, то в обучении без учителя система старается самостоятельно найти шаблоны непосредственно из приведенного примера.

Методы кластеризации данных являются одним из наиболее популярных семейств машинного обучения без учителя. Рассмотрим некоторые из них подробнее. Начнем с иерархиеской кластеризации.

## Иерархическая кластеризация 

Иерархическая кластеризация (также графовые алгоритмы кластеризации и иерархический кластерный анализ) — совокупность алгоритмов упорядочивания данных, направленных на создание иерархии (дерева) вложенных кластеров. Выделяют два класса методов иерархической кластеризации:

- Агломеративные методы (англ. agglomerative): новые кластеры создаются путем объединения более мелких кластеров и, таким образом, дерево создается от листьев к стволу;
- Дивизивные или дивизионные методы (англ. divisive): новые кластеры создаются путем деления более крупных кластеров на более мелкие и, таким образом, дерево создается от ствола к листьям.

Алгоритмы иерархической кластеризации предполагают, что анализируемое множество объектов характеризуется определённой степенью связности. По количеству признаков иногда выделяют монотетические и политетические методы классификации. Как и большинство визуальных способов представления зависимостей графы быстро теряют наглядность при увеличении числа кластеров.

Выполним построение дендрограммы методом агломеративной кластеризации.

```python
X = df[col] #какие данные возьмем для кластеризации
#Желательно брать данные после нормализации данных

from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):
    # создадим матрицу связей для построения дендрограммы

    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Передаем данные для построения дендрограммы
    dendrogram(linkage_matrix, **kwargs)

# устанавливаем distance_threshold=0 чтиобы гарантированно посчитать полное дерево
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(X)
plt.figure(figsize=(15,5)) #размер фигуры
plt.title('Hierarchical Clustering Dendrogram')
# можно установить количество уровней дендрограммы, параметр р
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Количество точек в узле (или индекс точки, если нет скобок).")
plt.show()
```
Данная модель наглядна, но на практике чаще используют другой подход.

```python
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

Z = linkage(X, method='ward') #другие методы {“ward”, “complete”, “average”, “single”}, default=”ward”
plt.figure(figsize=(15,7))
dendrogram(Z, truncate_mode='level')
plt.show()
```

Нам нужна матрица linkage (связей).

```python
# максимизируем количество кластеров (параметр задаем)
max_clusters=fcluster(Z, 10, criterion='maxclust')
max_clusters[:10]
```

```python
#используем в качестве критерия расстояние
d_clusters=fcluster(Z,  t=50000, criterion='distance')
d_clusters[:10]
```

Не всегда просто интерпретировать полученные кластеры.

```python
df_result=df[col].copy()
df_result['max_clusters']=max_clusters
df_result['distance']=d_clusters
df_result.sample(10)
```

Выполним группировку по кластерам.
```python
df_analize=df_result.groupby('max_clusters')[col].mean()
df_analize['Count']=df_result.groupby('max_clusters')['max_clusters'].count()
df_analize
```

Также полезно визуализировать результаты.

```python
import seaborn as sns
import matplotlib.pyplot as plt
 
x=col[0] #Изменяйте столбцы 
y=col[2]
print(x,y)
sns.lmplot( x=x, y=y, data=df_result, fit_reg=False, hue='max_clusters', legend=False)
plt.legend(loc='lower right')
plt.show()
```

Лучше работает при небольшом наборе кластеров.

```python
g = sns.lmplot(x=x, y=y, hue="max_clusters", col="max_clusters",
               data=df_result, height=6, aspect=.4, x_jitter=.1)
```

Есть вариант в две колонки

```python
g = sns.lmplot(x=x, y=y, hue="max_clusters", col="max_clusters",
               data=df_result, col_wrap=2, height=3)
```

## Кластеризация методом Kmeans

Иерархическая кластеризация хуже подходит для кластеризации больших объемов данных в сравнении с методом k-средних. Это объясняется тем, что временная сложность алгоритма линейна для метода k-средних (O(n)) и квадратична для метода иерархической кластеризации (O(n2)).

В кластеризации при помощи метода k-средних алгоритм начинает построение с произвольного выбора начальных точек, поэтому результаты, генерируемые при многократном запуске алгоритма, могут отличаться. В то же время в случае иерархической кластеризации результаты воспроизводимы.

Из центроидной геометрии построения метода k-средних следует, что метод хорошо работает, когда форма кластеров является гиперсферической (например, круг в 2D или сфера в 3D).

Метод k-средних более чувствителен к зашумленным данным, чем иерархический метод.

### Метод локтя

Если истинная метка заранее не известна(как в вашем случае), то K-Means clustering можно оценить с помощью критерия локтя или коэффициента силуэта.

Идея метода локтя состоит в том , чтобы выполнить кластеризацию k-средних по заданному набору данных для диапазона значений k ( num_clusters, например k=1-10) и для каждого значения k вычислить сумму квадратов ошибок (SSE).

После этого постройте линейный график SSE для каждого значения k. Мы хотим свести к минимуму SSE. SSE имеет тенденцию уменьшаться к 0, когда мы увеличиваем k (и SSE равно 0, когда k равно числу точек данных в наборе данных, потому что тогда каждая точка данных является своим собственным кластером, и нет никакой ошибки между ней и центром ее кластера).

Таким образом, цель состоит в том, чтобы выбрать наименьшее значение k , который все еще имеет низкий SSE, и локоть обычно представляет, где мы начинаем иметь убывающую отдачу при увеличении k.

```python
from sklearn.cluster import KMeans

X=df[col].values ### переменую после нормирования

sse = {}
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Количество кластеров")
plt.ylabel("SSE")
plt.show()
```

### Метод Коэффициента Силуэта:

Более высокая оценка коэффициента силуэта относится к модели с более четко определенными кластерами. Коэффициент силуэта определяется для каждой выборки и состоит из двух баллов:
- a: среднее расстояние между образцом и всеми другими точками того же класса. 
- b: среднее расстояние между образцом и всеми другими точками в следующей точке ближайший кластер.

Коэффициент силуэта для одного образца затем задается как:

s=b-a/max(a,b)

Теперь, чтобы найти оптимальное значение k для KMeans, выполните цикл через 1..n для n_clusters в KMeans и вычислите коэффициент силуэта для каждой выборки.

Более высокий коэффициент силуэта указывает на то, что объект хорошо подобран к своему собственному кластеру и плохо подобран к соседним кластерам.

```python
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

for n_cluster in range(2, 11):
    kmeans = KMeans(n_clusters=n_cluster).fit(X)
    label = kmeans.labels_
    sil_coeff = silhouette_score(X, label, metric='euclidean')
    print("Для n_clusters={}, коэффициент силуэта {}".format(n_cluster, sil_coeff))
```

Пример выполнения кластеризации и визуализации с отображением центров кластеров.

```python
kmeans = KMeans(n_clusters = 4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:, 0], X[:, 1], c = y_kmeans, s = 20, cmap = 'summer')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c = 'blue', s = 100, alpha = 0.9);
plt.show()
```

Обработка результатов происходит также как и в случае иерархической кластеризации. 

## DBSCAN

DBSCAN (Density-based spatial clustering of applications with noise, плотностной алгоритм пространственной кластеризации с присутствием шума), как следует из названия, оперирует плотностью данных. На вход он просит уже знакомую матрицу близости и два параметра — радиус -окрестности и количество соседей. 
![Классическая картинка](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png)


[Описание алгоритмов](https://scikit-learn.org/stable/modules/clustering.html)

[Подробнее](https://habr.com/ru/post/322034/)

Реализация алгоритма крайне проста с использованием библиотек. 
```python
X=df[col] #заменяем переменную

from sklearn.cluster import DBSCAN

# eps - радиус
# минимальное количество участников в кластере
db = DBSCAN(eps=1, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
db.labels_
```

Отрисуем график.

```python
colors = ['royalblue', 'maroon', 'forestgreen', 'mediumorchid', 'tan', 'deeppink', 'olive', 'goldenrod', 'lightcyan', 'navy']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])
plt.scatter(X[:,0], X[:,2], c=vectorizer(db.labels_))
```

Чтобы получить количество кластеров, надо выполнить `db.labels_.max()`.

Обработка результатов проходит аналогично другим видам кластеризации. 


# Снижение размерности

Одной из самых очевидных задач, которые возникают в отсутствие явной разметки, является задача снижения размерности данных. С одной стороны её можно рассматривать как помощь в визуализации данных, c другой стороны подобное снижение размерности может убрать лишние сильно скоррелированные признаки у наблюдений и подготовить данные для дальнейшей обработки в режиме обучения с учителем, например сделать входные данные более "перевариваемыми" для деревьев решений.

Объемы и сложность данных постоянно растут. В результате, существенно увеличивается и их размерность. Для компьютеров это не проблема — в отличие от людей: мы ограничены всего тремя измерениями.

Структура, скрытая в данных, может быть восстановлена только с помощью специальных математических методов. К ним относится подраздел машинного обучения без учителя под названием множественное обучение (manifold learning) или нелинейное уменьшение размерности (nonlinear dimensionality reduction).

## PCA

PCA помогает выразить несколько признаков через один, что позволяет работать с более простой моделью.

PCA аппроксимирует n-размерное облако наблюдений до эллипсоида (тоже n-мерного), полуоси которого и будут являться будущими главными компонентами. И при проекции на такие оси (снижении размерности) сохраняется наибольшее количество информации.

Вычисление главных компонент сводится к вычислению собственных векторов и собственных значений ковариационной матрицы исходных данных или к сингулярному разложению матрицы данных.

```python
from sklearn import decomposition

pca = decomposition.PCA(n_components=2)
X_pca = pca.fit_transform(df[col])

x_axis = X_pca[:, 0]
y_axis = X_pca[:, 1]

plt.scatter(x_axis, y_axis, c=df_result.max_clusters)
plt.show()
```

Можно посмотреть, как образованы главные компоненты.

```python
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1], ["Первая компонента", "Вторая компонента"])
plt.colorbar()
plt.xticks(range(col), col, rotation=60, ha='left')
plt.xlabel("Характеристика")
plt.ylabel("Главные компоненты")
```

# t-SNE

t-SNE (t-distributed stochastic neighbor embedding) — техника нелинейного снижения размерности и визуализации многомерных переменных. Этот алгоритм может свернуть сотни измерений к меньшему количеству, сохраняя при этом важные отношения между данными: чем ближе объекты располагаются в исходном пространстве, тем меньше расстояние между этими объектами в пространстве сокращенной размерности. t-SNE неплохо работает на маленьких и средних реальных наборах данных и не требует большого количества настроек гиперпараметров.

Алгоритм t-SNE, который также относят к методам множественного обучения признаков, был опубликован в 2008 году голландским исследователем Лоуренсом ван дер Маатеном (сейчас работает в Facebook AI Research) и Джеффри Хинтоном. Классический SNE был предложен Хинтоном и Ровейсом в 2002. В статье 2008 года описывается несколько «трюков», которые позволили упростить процесс поиска глобальных минимумов, и повысить качество визуализации.

```python
from sklearn.manifold import TSNE

# Определяем модель и скорость обучения
model = TSNE(learning_rate=100)

# Обучаем модель
transformed = model.fit_transform(rob)

x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

plt.scatter(x_axis, y_axis, c=df_result.max_clusters)
plt.show()
```

# Ассоциативные правила

Ассоциативные правила позволяют находить закономерности между связанными событиями. Примером такой закономерности служит правило, указывающее, что из события X следует событие Y с некоторой вероятностью. Установление таких зависимостей дает возможность находить очень простые и интуитивно понятные правила.

```python
#прочитаем таблицу
with engine.connect() as session:
    df=pd.read_sql('SELECT * FROM grdp LIMIT 1000', con=session)
```

Проведем подготовительные работы.

```python
#выделю колонки, которые хочу проанализировать
col=['Продолжительность', 'Характер', 'Телеграмма', 'Деффект']
#удалим все значение None
df.dropna(subset=col, axis='index', inplace=True)
# выполним приведение типа к строковому
df['Продолжительность']=df['Продолжительность'].astype(str)
#сформирую список событий
transactions=df[col].values
```

Выполним сам расчет.

```python
# загрузим пакеты, необходимые для выполнения анализа
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_as = pd.DataFrame(te_ary, columns=te.columns_)
#параметры можно регулировать, например, сразу отсечь все мало встречаемое
frequent_itemsets = apriori(df_as, min_support=0.2, use_colnames=True)
```

И выведем результаты c фильстром по уровню поддержки

```python
# сгенерируем ассоциативные правила с уровнем доверия 0.1 Стоит учитывать, что данный уровень поддержки крайне
# низкий и используется только для примера
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
```
С фильтром по уровню независимости. 

```python
# найдем ассоциативные правила с уровнем независимости больше 1.2
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules
```

Расшифровка важных для нас параметров:
- antecedents - посыл
- consequents - последствия
- antecedent support - встречаемость посыла
- consequent support - встречаемость последствия
- support - совместная встречаемость
- confidence - вероятность появления последствия при наличии посыла
- lift - условно мера случайности. Если значение меньше единицы, правилу доверять не стоит
- leverage - это разность между наблюдаемой частотой, с которой условие и
следствие появляются совместно (т.е., поддержкой ассоциации), и
произведением частот появления (поддержек) условия и следствия по
отдельности. 
- Conviction - В общем виде Conviction — это «частотность ошибок» нашего правила. Должно быть больше 1.